In [1]:
from pyspark.sql import SparkSession

spark_session = SparkSession.builder\
        .master("spark://192.168.2.97:7077") \
        .appName("Q3")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.4.0-3.3")\
        .config("spark.cores.max", 2)\
        .getOrCreate()

spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
uk.co.gresearch.spark#spark-extension_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-be06fd17-3a21-41b1-8ad2-47f6e3a41182;1.0
	confs: [default]
	found uk.co.gresearch.spark#spark-extension_2.12;2.4.0-3.3 in central
downloading https://repo1.maven.org/maven2/uk/co/gresearch/spark/spark-extension_2.12/2.4.0-3.3/spark-extension_2.12-2.4.0-3.3.jar ...
	[SUCCESSFUL ] uk.co.gresearch.spark#spark-extension_2.12;2.4.0-3.3!spark-extension_2.12.jar (58ms)
:: resolution report :: resolve 3150ms :: artifacts dl 67ms
	:: modules in use:
	uk.co.gresearch.spark#spark-extension_2.12;2.4.0-3.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------

In [2]:
redComm1 = spark_session.read.json("../RC_2008-03.json")
redComm2 = spark_session.read.json("../RC_2008-04.json")

In [5]:
redComm1.take(5)

AnalysisException: 
Since Spark 2.3, the queries from raw JSON/CSV files are disallowed when the
referenced columns only include the internal corrupt record column
(named _corrupt_record by default). For example:
spark.read.schema(schema).csv(file).filter($"_corrupt_record".isNotNull).count()
and spark.read.schema(schema).csv(file).select("_corrupt_record").show().
Instead, you can cache or save the parsed results and then send the same query.
For example, val df = spark.read.schema(schema).csv(file).cache() and then
df.filter($"_corrupt_record".isNotNull).count().
      

In [4]:
subs = redComm1.select('subreddit')
subs.take(5)

AnalysisException: cannot resolve 'subreddit' given input columns: [_corrupt_record];
'Project ['subreddit]
+- Relation [_corrupt_record#7] json


In [ ]:
subCountM1 = subs\
    .groupBy("subreddit")\
    .count()\
    .orderBy("count", ascending=False)

In [ ]:
subCountM2 = redComm2\
    .select('subreddit')\
    .groupBy('subreddit')\
    .count()\
    .orderBy('count', ascending=False)

In [ ]:
subCountM1.show()

In [ ]:
subCountM2.join(subCountM1,subCountM2.subreddit ==  subCountM1.subreddit,"left")\
    .show(truncate=False)
subCountM2.join(subCountM1,subCountM2.subreddit ==  subCountM1.subreddit,"leftouter")\
    .show(truncate=False)

In [ ]:
spark_session.stop()